Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 4.99 ms, sys: 0 ns, total: 4.99 ms
Wall time: 11.8 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.86/10389/1 Dashboard: http://10.20.0.86:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 215 ms, sys: 23.3 ms, total: 238 ms
Wall time: 2.01 s


[1.5932676971606397,
 1.2300522927656454,
 1.7343707569318427,
 1.424468548981303,
 1.0309922951822217,
 1.7739794709828822,
 1.1560868184008868,
 1.3983327767858114,
 1.9789931897985125,
 1.2546276410611954,
 1.5550413701792127,
 1.3952077705886776,
 1.3588763574654408,
 1.6727619970224863,
 1.5071871434108886,
 1.380898209574088,
 1.6901444751877777,
 1.7949026204014313,
 1.8199787724055487,
 1.3993173509503254,
 1.7010026291635545,
 1.5950174167535698,
 1.0944375138550178,
 1.3472519745250022,
 1.1755653874361678,
 1.368770974205575,
 1.155362078465112,
 1.3029387361926092,
 1.3347842851090062,
 1.5378335371021659,
 1.440254976615883,
 1.8959687474509983,
 1.8965344438031075,
 1.6718635331103209,
 1.3745619286000665,
 1.4830005050820314,
 1.7067337406961023,
 1.6188176727642978,
 1.610386758249006,
 1.9054714940450719,
 1.0222370590349499,
 1.1199435202841017,
 1.8610955885416396,
 1.5590454456878788,
 1.6958983798177028,
 1.7847378122737367,
 1.2245319378864723,
 1.5946396350013439

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.011942590027322,
 1.3461347644107602,
 1.1294350884143407,
 1.8690214351907557,
 1.8196961751532559,
 1.8805240876183764,
 1.2934092084548179,
 1.171328479117744,
 1.1694798991432604,
 1.5426605663483435,
 1.2495795959614298,
 1.4764486124617269,
 1.6222037004054308,
 1.0173858497518715,
 1.5868145422576525,
 1.8296947039495568,
 1.0380143797676684,
 1.0846780951945456,
 1.8480704443603413,
 1.5658838577401286,
 1.4228133740200426,
 1.8496478186457472,
 1.4179912287148904,
 1.1677002859351944,
 1.515501164902533,
 1.7143238151662339,
 1.184217721062523,
 1.0374524825997349,
 1.9637357150616361,
 1.9116364682255536,
 1.0975834266471027,
 1.4873823422237107,
 1.184398413056476,
 1.1196525750927662,
 1.1218102741756328,
 1.2710192986544184,
 1.7659090994097393,
 1.894272312946804,
 1.1005118423809606,
 1.7279863586433983,
 1.1378074590116363,
 1.2645593801934476,
 1.8452059154520333,
 1.9722897705413471,
 1.4677465809033476,
 1.2298330792034435,
 1.9617865543398878,
 1.6145492780478703

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 156 ms, sys: 27.5 ms, total: 184 ms
Wall time: 2.31 s
